In [2]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.stats import norm


def black_scholes_call(S, K, T, r, sigma):
  """
  Calculates the theoretical price of a European call option using Black-Scholes formula.

  Args:
      S: Current stock price
      K: Strike price of the option
      T: Time to expiration in years
      r: Risk-free interest rate
      sigma: Volatility of the underlying asset

  Returns:
      Theoretical call option price
  """
  d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [2]:
from dataclasses import dataclass
from typing import Dict, List
from scipy.optimize import root_scalar
from scipy.stats import norm

import json  # Assuming you use json for reading config

# Read configuration from JSON file
with open("config.json") as f:
  config = json.load(f)
r = config["risk_free_interest_rate"]
nt = config["number_of_trading_days"]
upper_delta = config["upper_delta"]
lower_delta = config["lower_delta"]


@dataclass
class Option:
  """
  Represents an option contract with relevant data.
  """
  strike: float
  premium: float
  delta: float
  theta: float
  underlying_price: float


@dataclass
class StockAnalysis:
  """
  Performs analysis of a stock and its options for writing put options.
  """
  historical_data: pd.DataFrame  # Replace with actual import for pandas
  current_price: float

  def calculate_annualized_volatility(self):
    """
    Calculates annualized volatility based on daily returns.
    """
    daily_returns = self.historical_data['Close'].pct_change()
    return daily_returns.std() * np.sqrt(nt)

  def black_scholes_call(self, K, T, sigma):
    """
    Calculates the theoretical price of a European call option using Black-Scholes formula.
    """
    d1 = (np.log(self.current_price / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return self.current_price * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

  def implied_volatility_atm_put(self, option, T):
    """
    Estimates implied volatility for an at-the-money (ATM) put option using Black-Scholes.

    Args:
        option: Option data (dictionary)
        T: Time to expiration in years

    Returns:
        Estimated implied volatility or None if ATM put not found.
    """
    K = option['strike']
    P = option['premium']

    def call_put_parity(sigma):
      # Call put parity - Call price - Put price - K + S*e^(-r*T) = 0
      call_price = self.black_scholes_call(K, T, sigma)
      return call_price - P - K + self.current_price * np.exp(-r * T)

    # Find root of call-put parity for ATM put (put price = call price)
    try:
      sol = root_scalar(call_put_parity, bracket=(0.01, 1.0))
      return sol.root
    except Exception:
      return None  # Handle potential errors or no ATM put found

  def analyze_put_options(self, options: List[Dict]) -> List[Dict]:
    """
    Analyzes put options and identifies potentially profitable ones for writing.

    Args:
        options: List of option dictionaries

    Returns:
        List of dictionaries containing promising put options with additional data.
    """
    promising_puts = []
    for option in options:
      strike_price = option['strike']
      premium = option['premium']
      delta = option['delta']
      theta = option['theta']

      # Calculate time to expiration in years
      T = option['days_to_expiry'] / (nt * 365.25)  # Assuming 365.25 days in a year


      annualized_volatility = self.calculate_annualized_volatility()
      implied_volatility = self.implied_volatility_atm_put(option, T)
      if not implied_volatility:
        implied_volatility = annualized_volatility

      max_profit = premium
      max_loss = self.current_price - 3 * implied_volatility
      max_loss = max(max_loss, 0)  # Enforce positive max loss
      profit_loss_ratio = max_profit / max_loss

      if lower_delta < delta < upper_delta:
        promising_puts.append({
            "strike": strike_price,
            "premium": premium,
            "delta": delta,
            "theta": theta,
            "profit_loss_ratio": profit_loss_ratio,
            "annualized_volatility": annualized_volatility,
            "implied_volatility": implied_volatility  # Include if available
      })

    # Sort promising puts by desired criteria (e.g., profit-loss ratio)
    promising_puts.sort(key=lambda p: p['profit_loss_ratio'], reverse=True)

    return promising_puts


FileNotFoundError: [Errno 2] No such file or directory: 'config.json'